# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,seven pagodas,12.6264,80.1722,25.56,77,34,3.66,IN,1729895690
1,1,jamestown,42.0970,-79.2353,12.80,66,75,2.57,US,1729895691
2,2,college,64.8569,-147.8028,-4.75,80,20,0.45,US,1729895692
3,3,hawaiian paradise park,19.5933,-154.9731,24.94,98,100,1.54,US,1729895693
4,4,iqaluit,63.7506,-68.5145,-1.15,80,75,4.63,CA,1729895694


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
import hvplot.pandas
import pandas as pd

# Load the data (assuming city_data_df is already loaded as per instructions)
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat',                      # Longitude and Latitude for x and y
    geo=True,                          # Enables geographic plotting
    tiles='OSM',                       # Use OpenStreetMap tiles as the background
    size='Humidity',                   # Size of each point is proportional to humidity
    color='City',                      # Unique color for each city
    legend='right',                    # Places the legend on the right side to match image
    frame_width=800,                   # Width of the plot frame
    frame_height=500,                  # Height of the plot frame
    title='City Locations with Humidity Represented by Point Size',
    fill_alpha=0.4,                    # Set transparency level for see-through effect
    xlabel='x',                        # Set minimal label for x-axis
    ylabel='y',                        # Set minimal label for y-axis
    
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Example criteria: cities with Max Temp between 20 and 30, Humidity below 60, and Cloudiness below 20

ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) &  # Temperature range
    (city_data_df['Humidity'] < 60) &                                      # Low humidity
    (city_data_df['Cloudiness'] < 20)                                      # Low cloudiness
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(20)  # Displaying the first 20 rows to match the output sample


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,fort bragg,35.1390,-79.0060,23.91,39,0,2.57,US,1729895679
61,61,concepcion del oro,24.6333,-101.4167,20.68,12,0,4.63,MX,1729895759
89,89,galliano,29.4422,-90.2992,26.00,57,0,0.00,US,1729895793
124,124,crane,31.3974,-102.3501,25.74,28,6,4.22,US,1729895832
127,127,mount isa,-20.7333,139.5000,25.87,10,3,4.12,AU,1729895837
142,142,guerrero negro,27.9769,-114.0611,22.91,51,6,6.08,MX,1729895855
224,224,evans,33.5337,-82.1307,26.69,47,4,0.45,US,1729895861
230,230,fortuna,40.5982,-124.1573,22.21,32,0,4.13,US,1729895960
264,264,bahadurgarh,28.6833,76.9167,21.06,25,0,1.61,IN,1729896004
265,265,sangin,32.0728,64.8359,21.17,31,6,1.14,AF,1729896005


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create a DataFrame called hotel_df
# This will store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name", to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(20)  # Display the first 20 rows to match the sample output


,City,Country,Lat,Lng,Humidity,Hotel Name
23,fort bragg,US,35.1390,-79.0060,39,
61,concepcion del oro,MX,24.6333,-101.4167,12,
89,galliano,US,29.4422,-90.2992,57,
124,crane,US,31.3974,-102.3501,28,
127,mount isa,AU,-20.7333,139.5000,10,
142,guerrero negro,MX,27.9769,-114.0611,51,
224,evans,US,33.5337,-82.1307,47,
230,fortuna,US,40.5982,-124.1573,32,
264,bahadurgarh,IN,28.6833,76.9167,25,
265,sangin,AF,32.0728,64.8359,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Search radius in meters
params = {
    "categories": "accommodation.hotel",  # Category for hotels
    "limit": 1,                           # We only need the first (nearest) hotel
    "apiKey": geoapify_key                # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get Latitude, Longitude from the DataFrame
    lat, lon = row['Lat'], row['Lng']
    
    # Add the current city's Latitude and Longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
concepcion del oro - nearest hotel: No hotel found
galliano - nearest hotel: Fairfield Inn & Suites Cut Off-Galliano
crane - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
guerrero negro - nearest hotel: Plaza sal paraiso
evans - nearest hotel: Hyatt Place Augusta
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
bahadurgarh - nearest hotel: Motel Gajraj Continental
sangin - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
marsa alam - nearest hotel: Blue House Hotel
chaves - nearest hotel: Home2 Suites by Hilton Roswell, NM
valparaiso - nearest hotel: Ibis Hotel
reggane - nearest hotel: No hotel found
siavonga - nearest hotel: Lake Safari Lodge Annex
alice springs - nearest hotel: Aurora Alice Springs
lompoc - nearest hotel: Red Roof Inn Lompoc
alpine - nearest hotel: Ayres Lodge Alpine
swainsboro - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
23,fort bragg,US,35.1390,-79.0060,39,Airborne Inn Lodging
61,concepcion del oro,MX,24.6333,-101.4167,12,No hotel found
89,galliano,US,29.4422,-90.2992,57,Fairfield Inn & Suites Cut Off-Galliano
124,crane,US,31.3974,-102.3501,28,No hotel found
127,mount isa,AU,-20.7333,139.5000,10,Ibis Styles
142,guerrero negro,MX,27.9769,-114.0611,51,Plaza sal paraiso
224,evans,US,33.5337,-82.1307,47,Hyatt Place Augusta
230,fortuna,US,40.5982,-124.1573,32,Comfort Inn & Suites Redwood Country
264,bahadurgarh,IN,28.6833,76.9167,25,Motel Gajraj Continental
265,sangin,AF,32.0728,64.8359,31,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot with hover information formatted to match the example
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',                            # Longitude and Latitude for x and y
    geo=True,                                # Enables geographic plotting
    tiles='OSM',                             # Use OpenStreetMap tiles as the background
    size='Humidity',                         # Size of each point is proportional to humidity
    color='City',                            # Unique color for each city
    legend='right',                          # Places the legend on the right side
    frame_width=800,                         # Width of the plot frame
    frame_height=500,                        # Height of the plot frame
    fill_alpha=0.4,                          # Set point transparency to semi-transparent
    xlabel='x',                              # Minimal label for x-axis
    ylabel='y' ,                              # Minimal label for y-axis
    hover_cols=['Hotel Name', 'Country']
)



# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)